# Build the "HomeMatch" application

## Step 1: Setting Up the Python Application

In [1]:
import os
import re
import csv
import lancedb
import pandas as pd
import numpy as np
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from lancedb.pydantic import vector, LanceModel

openai_api_key = os.getenv("OPENAI_API_KEY")

## Step2: Generating Real Estate Listings

In [40]:
listings = [
    {
        "Neighborhood": "Green Oaks",
        "Price": "800,000",
        "Bedrooms": 3,
        "Bathrooms": 2,
        "House Size": "2,000 sqft"
    }
]

prompt_template = """
Please generate 10 real estate listings for a property with the following details:
Neighborhood:
Price:
Bedrooms:
Bathrooms:
House Size:
Description:
Neighborhood Description:
"""

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)
    

prompt = PromptTemplate(
    template=prompt_template
)

chain = LLMChain(
    llm=llm,
    prompt=prompt
)


response = chain.run(listings[0])
print(response)

pattern = re.compile(
    r"Neighborhood: (.*?)\n"
    r"Price: (.*?)\n"
    r"Bedrooms: (\d+)\n"
    r"Bathrooms: ([\d.]+)\n"
    r"House Size: (.*?)\n"
    r"Description: (.*?)\n"
    r"Neighborhood Description: (.*?)(?=\n\d+\.|$)",
    re.DOTALL
)

matches = pattern.findall(response)

listings = []
for match in matches:
    listing = {
        "neighborhood": match[0].strip(),
        "price": int(match[1].strip().replace("$", "").replace(",", "")),
        "bedrooms": int(match[2].strip()),
        "bathrooms": float(match[3].strip()),
        "house_size": int(match[4].strip().replace("sqft", "").replace(",","")),
        "description": match[5].strip(),
        "neighborhood_description": match[6].strip()
    }
    listings.append(listing)

for listing in listings:
    print(listing)

# Save to CSV
csv_file = "listings.csv"
csv_columns = ["house_id", "neighborhood", "price", "bedrooms", "bathrooms", "house_size", "description", "neighborhood_description", "full_description"]

try:
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=csv_columns)
        writer.writeheader()
        for i,listing in enumerate(listings):
            listing["house_id"] = i + 1
            listing["full_description"] = f"price: {listing["price"]}, bedrooms: {listing["bedrooms"]}, bathrooms: {listing["bathrooms"]}, house_size: {listing["house_size"]}, full_description{listing["description"] + listing["neighborhood_description"]}"
            writer.writerow(listing)
    print(f"Listings have been saved to {csv_file}")
except IOError:
    print("I/O error")

1. Neighborhood: Upper East Side, Manhattan, NY
Price: $2,500,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: Stunning luxury apartment with high-end finishes and breathtaking views of Central Park.
Neighborhood Description: Upscale neighborhood known for its exclusive shops, fine dining, and proximity to Central Park.

2. Neighborhood: Venice Beach, Los Angeles, CA
Price: $1,200,000
Bedrooms: 2
Bathrooms: 1
House Size: 1,500 sqft
Description: Charming beach bungalow with a private backyard and just steps away from the ocean.
Neighborhood Description: Trendy neighborhood known for its bohemian vibe, eclectic shops, and vibrant street art scene.

3. Neighborhood: Buckhead, Atlanta, GA
Price: $800,000
Bedrooms: 4
Bathrooms: 3
House Size: 3,500 sqft
Description: Spacious family home with a gourmet kitchen, large backyard, and a bonus room.
Neighborhood Description: Upscale neighborhood known for its luxury shopping, fine dining, and prestigious schools.

4. Neighborhood: 

## Step3: Storing Listings in a Vector Database

In [41]:
df = pd.read_csv("./listings.csv", header=0)
df = df.set_index("house_id").reindex(df.house_id)
df

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description,full_description
house_id,,,,,,,,
1,"Upper East Side, Manhattan, NY",2500000,3,2.0,2000,Stunning luxury apartment with high-end finish...,Upscale neighborhood known for its exclusive s...,"price: 2500000, bedrooms: 3, bathrooms: 2.0, h..."
2,"Venice Beach, Los Angeles, CA",1200000,2,1.0,1500,Charming beach bungalow with a private backyar...,Trendy neighborhood known for its bohemian vib...,"price: 1200000, bedrooms: 2, bathrooms: 1.0, h..."
3,"Buckhead, Atlanta, GA",800000,4,3.0,3500,"Spacious family home with a gourmet kitchen, l...",Upscale neighborhood known for its luxury shop...,"price: 800000, bedrooms: 4, bathrooms: 3.0, ho..."
4,"South Beach, Miami, FL",1500000,1,1.5,1200,"Modern condo with ocean views, high ceilings, ...",Vibrant neighborhood known for its beautiful b...,"price: 1500000, bedrooms: 1, bathrooms: 1.5, h..."
5,"Pacific Heights, San Francisco, CA",3000000,5,4.0,4000,Elegant Victorian home with a landscaped garde...,Prestigious neighborhood known for its histori...,"price: 3000000, bedrooms: 5, bathrooms: 4.0, h..."
6,"Georgetown, Washington, D.C.",1800000,3,2.5,2500,"Renovated townhouse with a private patio, gour...",Historic neighborhood known for its cobbleston...,"price: 1800000, bedrooms: 3, bathrooms: 2.5, h..."
7,"Old Town, Alexandria, VA",900000,2,2.0,1800,"Quaint historic home with a fenced backyard, h...",Charming neighborhood known for its colonial a...,"price: 900000, bedrooms: 2, bathrooms: 2.0, ho..."
8,"Cherry Creek, Denver, CO",1100000,4,3.0,2800,"Modern townhouse with a rooftop deck, open flo...",Trendy neighborhood known for its upscale shop...,"price: 1100000, bedrooms: 4, bathrooms: 3.0, h..."
9,"The Woodlands, Houston, TX",700000,4,3.5,3000,"Spacious family home with a large backyard, go...",Master-planned community known for its wooded ...,"price: 700000, bedrooms: 4, bathrooms: 3.5, ho..."


In [42]:
embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
documents = [Document(content) for content in df['full_description']]
documents

[Document(page_content='price: 2500000, bedrooms: 3, bathrooms: 2.0, house_size: 2000, full_descriptionStunning luxury apartment with high-end finishes and breathtaking views of Central Park.Upscale neighborhood known for its exclusive shops, fine dining, and proximity to Central Park.'),
 Document(page_content='price: 1200000, bedrooms: 2, bathrooms: 1.0, house_size: 1500, full_descriptionCharming beach bungalow with a private backyard and just steps away from the ocean.Trendy neighborhood known for its bohemian vibe, eclectic shops, and vibrant street art scene.'),
 Document(page_content='price: 800000, bedrooms: 4, bathrooms: 3.0, house_size: 3500, full_descriptionSpacious family home with a gourmet kitchen, large backyard, and a bonus room.Upscale neighborhood known for its luxury shopping, fine dining, and prestigious schools.'),
 Document(page_content='price: 1500000, bedrooms: 1, bathrooms: 1.5, house_size: 1200, full_descriptionModern condo with ocean views, high ceilings, and 

In [43]:
embeddings = embeddings_model.embed_documents([doc.page_content for doc in documents])
len(embeddings)

10

In [44]:
df["vector"] = embeddings
df

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description,full_description,vector
house_id,,,,,,,,,
1,"Upper East Side, Manhattan, NY",2500000,3,2.0,2000,Stunning luxury apartment with high-end finish...,Upscale neighborhood known for its exclusive s...,"price: 2500000, bedrooms: 3, bathrooms: 2.0, h...","[0.003238660063478017, 0.020455370199525723, 0..."
2,"Venice Beach, Los Angeles, CA",1200000,2,1.0,1500,Charming beach bungalow with a private backyar...,Trendy neighborhood known for its bohemian vib...,"price: 1200000, bedrooms: 2, bathrooms: 1.0, h...","[0.0010383232336779727, 0.026649475952326078, ..."
3,"Buckhead, Atlanta, GA",800000,4,3.0,3500,"Spacious family home with a gourmet kitchen, l...",Upscale neighborhood known for its luxury shop...,"price: 800000, bedrooms: 4, bathrooms: 3.0, ho...","[0.006494032247863247, 0.031092440847414072, -..."
4,"South Beach, Miami, FL",1500000,1,1.5,1200,"Modern condo with ocean views, high ceilings, ...",Vibrant neighborhood known for its beautiful b...,"price: 1500000, bedrooms: 1, bathrooms: 1.5, h...","[-0.0035825361474121054, 0.02269479905645152, ..."
5,"Pacific Heights, San Francisco, CA",3000000,5,4.0,4000,Elegant Victorian home with a landscaped garde...,Prestigious neighborhood known for its histori...,"price: 3000000, bedrooms: 5, bathrooms: 4.0, h...","[0.008034458660677223, 0.0042579990415734775, ..."
6,"Georgetown, Washington, D.C.",1800000,3,2.5,2500,"Renovated townhouse with a private patio, gour...",Historic neighborhood known for its cobbleston...,"price: 1800000, bedrooms: 3, bathrooms: 2.5, h...","[-0.007964151958245005, 0.022478506101159857, ..."
7,"Old Town, Alexandria, VA",900000,2,2.0,1800,"Quaint historic home with a fenced backyard, h...",Charming neighborhood known for its colonial a...,"price: 900000, bedrooms: 2, bathrooms: 2.0, ho...","[-0.004987733520851104, 0.016754592113782888, ..."
8,"Cherry Creek, Denver, CO",1100000,4,3.0,2800,"Modern townhouse with a rooftop deck, open flo...",Trendy neighborhood known for its upscale shop...,"price: 1100000, bedrooms: 4, bathrooms: 3.0, h...","[-5.435705682646029e-05, 0.01781534710025636, ..."
9,"The Woodlands, Houston, TX",700000,4,3.5,3000,"Spacious family home with a large backyard, go...",Master-planned community known for its wooded ...,"price: 700000, bedrooms: 4, bathrooms: 3.5, ho...","[0.008407798794670922, 0.02501418223643001, -0..."


In [45]:
class RealEstate(LanceModel):
    neighborhood: str
    vector: vector(len(embeddings[0]))
    price: int
    bedrooms: int
    bathrooms: float
    house_size: int
    description: str
    neighborhood_description: str

db = lancedb.connect("~/.lancedb")
table_name = "real_estate"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, data=df)

In [46]:
table.count_rows()

10

In [47]:
table.to_pandas()

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description,full_description,vector
0,"Upper East Side, Manhattan, NY",2500000,3,2.0,2000,Stunning luxury apartment with high-end finish...,Upscale neighborhood known for its exclusive s...,"price: 2500000, bedrooms: 3, bathrooms: 2.0, h...","[0.00323866, 0.02045537, 0.0046775294, -0.0154..."
1,"Venice Beach, Los Angeles, CA",1200000,2,1.0,1500,Charming beach bungalow with a private backyar...,Trendy neighborhood known for its bohemian vib...,"price: 1200000, bedrooms: 2, bathrooms: 1.0, h...","[0.0010383233, 0.026649475, 0.011805481, 0.008..."
2,"Buckhead, Atlanta, GA",800000,4,3.0,3500,"Spacious family home with a gourmet kitchen, l...",Upscale neighborhood known for its luxury shop...,"price: 800000, bedrooms: 4, bathrooms: 3.0, ho...","[0.006494032, 0.03109244, -0.014928054, -0.012..."
3,"South Beach, Miami, FL",1500000,1,1.5,1200,"Modern condo with ocean views, high ceilings, ...",Vibrant neighborhood known for its beautiful b...,"price: 1500000, bedrooms: 1, bathrooms: 1.5, h...","[-0.0035825362, 0.022694798, 0.0026050387, -0...."
4,"Pacific Heights, San Francisco, CA",3000000,5,4.0,4000,Elegant Victorian home with a landscaped garde...,Prestigious neighborhood known for its histori...,"price: 3000000, bedrooms: 5, bathrooms: 4.0, h...","[0.008034458, 0.004257999, -0.0011840601, 0.00..."
5,"Georgetown, Washington, D.C.",1800000,3,2.5,2500,"Renovated townhouse with a private patio, gour...",Historic neighborhood known for its cobbleston...,"price: 1800000, bedrooms: 3, bathrooms: 2.5, h...","[-0.007964152, 0.022478506, 0.0072538885, -0.0..."
6,"Old Town, Alexandria, VA",900000,2,2.0,1800,"Quaint historic home with a fenced backyard, h...",Charming neighborhood known for its colonial a...,"price: 900000, bedrooms: 2, bathrooms: 2.0, ho...","[-0.0049877334, 0.016754592, -0.011770134, 0.0..."
7,"Cherry Creek, Denver, CO",1100000,4,3.0,2800,"Modern townhouse with a rooftop deck, open flo...",Trendy neighborhood known for its upscale shop...,"price: 1100000, bedrooms: 4, bathrooms: 3.0, h...","[-5.4357057e-05, 0.017815348, -0.0038351584, -..."
8,"The Woodlands, Houston, TX",700000,4,3.5,3000,"Spacious family home with a large backyard, go...",Master-planned community known for its wooded ...,"price: 700000, bedrooms: 4, bathrooms: 3.5, ho...","[0.008407799, 0.025014183, -0.026413303, 0.007..."
9,"North End, Boston, MA",1600000,3,2.0,2200,"Renovated brownstone with a private roof deck,...",Historic neighborhood known for its Italian he...,"price: 1600000, bedrooms: 3, bathrooms: 2.0, h...","[0.0014851596, 0.0166913, 0.007907781, -0.0051..."


## Step4: Building the User Preference Interface

In [48]:
query = "nice house with a view on the sea"
query_vector = embeddings_model.embed_documents([query])
results = table.search(query_vector[0]).limit(2).to_pydantic(RealEstate)
results

[RealEstate(neighborhood='South Beach, Miami, FL', vector=FixedSizeList(dim=1536), price=1500000, bedrooms=1, bathrooms=1.5, house_size=1200, description='Modern condo with ocean views, high ceilings, and access to a rooftop pool.', neighborhood_description='Vibrant neighborhood known for its beautiful beaches, trendy nightlife, and Art Deco architecture.'),
 RealEstate(neighborhood='Venice Beach, Los Angeles, CA', vector=FixedSizeList(dim=1536), price=1200000, bedrooms=2, bathrooms=1.0, house_size=1500, description='Charming beach bungalow with a private backyard and just steps away from the ocean.', neighborhood_description='Trendy neighborhood known for its bohemian vibe, eclectic shops, and vibrant street art scene.')]

In [49]:
def find(query, condition=""):
    query_vector = embeddings_model.embed_documents([query])
    if condition:
        return table.search(query_vector[0]).where(condition).to_pydantic(RealEstate)
    table.search(query_vector).where(condition).to_pydantic(RealEstate)
    

In [50]:
results = find("nice house with a view on the sea", "price > 2000000")
results

[RealEstate(neighborhood='Pacific Heights, San Francisco, CA', vector=FixedSizeList(dim=1536), price=3000000, bedrooms=5, bathrooms=4.0, house_size=4000, description='Elegant Victorian home with a landscaped garden, gourmet kitchen, and panoramic city views.', neighborhood_description='Prestigious neighborhood known for its historic mansions, stunning views of the Golden Gate Bridge, and upscale boutiques.'),
 RealEstate(neighborhood='Upper East Side, Manhattan, NY', vector=FixedSizeList(dim=1536), price=2500000, bedrooms=3, bathrooms=2.0, house_size=2000, description='Stunning luxury apartment with high-end finishes and breathtaking views of Central Park.', neighborhood_description='Upscale neighborhood known for its exclusive shops, fine dining, and proximity to Central Park.')]

In [53]:
results = find("nice house with a view on the sea", "price > 1000000 and bedrooms > 1")
results

[RealEstate(neighborhood='Venice Beach, Los Angeles, CA', vector=FixedSizeList(dim=1536), price=1200000, bedrooms=2, bathrooms=1.0, house_size=1500, description='Charming beach bungalow with a private backyard and just steps away from the ocean.', neighborhood_description='Trendy neighborhood known for its bohemian vibe, eclectic shops, and vibrant street art scene.'),
 RealEstate(neighborhood='Georgetown, Washington, D.C.', vector=FixedSizeList(dim=1536), price=1800000, bedrooms=3, bathrooms=2.5, house_size=2500, description='Renovated townhouse with a private patio, gourmet kitchen, and a fireplace.', neighborhood_description='Historic neighborhood known for its cobblestone streets, upscale shops, and waterfront dining options.'),
 RealEstate(neighborhood='Pacific Heights, San Francisco, CA', vector=FixedSizeList(dim=1536), price=3000000, bedrooms=5, bathrooms=4.0, house_size=4000, description='Elegant Victorian home with a landscaped garden, gourmet kitchen, and panoramic city views

## Step5: Searching Based on Preferences

In [54]:
questions = [
    "How big do you want your house to be?",
    "What are the 3 most important things for you in choosing this property?",
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?"
]


answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

def format_qa(questions, answers):
    formatted_qa = ""
    for q, a in zip(questions, answers):
        formatted_qa += f"Q: {q}\nA: {a}\n"
    return formatted_qa

formatted_qa = format_qa(questions, answers)

prompt_template = """
{formatted_qa}

Please generate the following details based on Q&As:
Neighborhood:
Price:
Bedrooms:
Bathrooms:
House Size:
Description:
Neighborhood Description:
"""

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature,
    openai_api_key=openai_api_key
)
    

prompt = PromptTemplate(
    template=prompt_template
)

chain = LLMChain(
    llm=llm,
    prompt=prompt
)


response = chain.run(formatted_qa)

pattern = re.compile(
    r"Neighborhood: (.*?)\n"
    r"Price: (.*?)\n"
    r"Bedrooms: (\d+)\n"
    r"Bathrooms: ([\d.]+)\n"
    r"House Size: (.*?)\n"
    r"Description: (.*?)\n"
    r"Neighborhood Description: (.*?)(?=\n\d+\.|$)",
    re.DOTALL
)

matches = pattern.findall(response)

preferences = []
for match in matches:
    preference = {
        "neighborhood": match[0].strip(),
        "price": match[1].strip(),
        "bedrooms": int(match[2].strip()),
        "bathrooms": float(match[3].strip()),
        "house_size": match[4].strip(),
        "description": match[5].strip(),
        "neighborhood_description": match[6].strip()
    }
    preferences.append(preference)

preferences[0]

{'neighborhood': 'Suburban neighborhood with access to urban amenities',
 'price': 'TBD based on market value',
 'bedrooms': 3,
 'bathrooms': 2.0,
 'house_size': 'Approximately 1,500 square feet',
 'description': 'This cozy three-bedroom house features a spacious kitchen, a cozy living room, a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. It is located in a quiet suburban neighborhood with good local schools and convenient shopping options. The house offers easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. It strikes a perfect balance between suburban tranquility and access to urban amenities like restaurants and theaters.',
 'neighborhood_description': 'The neighborhood is peaceful and family-friendly, with tree-lined streets and well-maintained homes. It offers a sense of community while also providing easy access to nearby urban amenities such as restaurants, theaters, and shopping centers. The area 

## Step 6: Personalizing Listing Descriptions

In [55]:
def get_full_description(pref):
        return f"price: {pref["price"]}, bedrooms: {pref["bedrooms"]}, bathrooms: {pref["bathrooms"]}, house_size: {pref["house_size"]}, full_description{pref["description"] + pref["neighborhood_description"]}"

def augment_customer_preference(customer_input):
    prompt_template = """
    {customer_input}
    
    Please generate the following details based on the customer_input. Please enter the price, bedrooms, bathrooms, and house size as a number.:
    Neighborhood:
    Price:
    Bedrooms:
    Bathrooms:
    House Size:
    Description:
    Neighborhood Description:
    """
    
    model_name = "gpt-3.5-turbo"
    temperature = 0.0
    openai_api_key = os.getenv("OPENAI_API_KEY")
    
    if not openai_api_key:
        raise ValueError("OPENAI_API_KEY environment variable not set")

    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature,
        openai_api_key=openai_api_key
    )
    
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["customer_input"]
    )
    
    chain = LLMChain(
        llm=llm,
        prompt=prompt
    )
    
    response = chain.run({"customer_input": customer_input})
    print("Generated Response:\n", response)
    
    pattern = re.compile(
        r"Neighborhood: (.*?)\n"
        r"Price: (.*?)\n"
        r"Bedrooms: (\d+)\+?\n"
        r"Bathrooms: ([\d.]+)\+?\n"
        r"House Size: (.*?)\n"
        r"Description: (.*?)\n"
        r"Neighborhood Description: (.*?)(?=\n|$)",
        re.DOTALL
    )
    
    matches = pattern.findall(response)

    preferences = []
    for match in matches:
        preference = {
            "neighborhood": match[0].strip(),
            "price": match[1].strip(),
            "bedrooms": int(match[2].strip()),
            "bathrooms": float(match[3].strip()),
            "house_size": match[4].strip(),
            "description": match[5].strip(),
            "neighborhood_description": match[6].strip()
        }
        preferences.append(preference)

    return preferences[0] if preferences else None

customer_input = """
I am looking for a comfortable four-bedroom house that can accommodate my family of five.
It should be in a safe neighborhood with good local schools. A large backyard is essential for my children to play in.
Additional amenities we need include a swimming pool, a two-car garage, and a home office.
The house should be in a suburban area with parks and recreational facilities nearby.
Easy access to public transportation and major highways is also important.
"""
pref = augment_customer_preference(customer_input)
pref

Generated Response:
 Neighborhood: Suburban area with parks and recreational facilities nearby
Price: $600,000
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sq ft
Description: Comfortable four-bedroom house with a large backyard, swimming pool, two-car garage, and home office.
Neighborhood Description: Safe neighborhood with good local schools, easy access to public transportation and major highways.


{'neighborhood': 'Suburban area with parks and recreational facilities nearby',
 'price': '$600,000',
 'bedrooms': 4,
 'bathrooms': 3.0,
 'house_size': '2,500 sq ft',
 'description': 'Comfortable four-bedroom house with a large backyard, swimming pool, two-car garage, and home office.',
 'neighborhood_description': 'Safe neighborhood with good local schools, easy access to public transportation and major highways.'}

In [56]:
def find_estate(pref):
    query = get_full_description(pref)
    query_vector = embeddings_model.embed_documents([query])
    results = table.search(query_vector[0]).limit(2).to_pydantic(RealEstate)
    return results

search_results = find_estate(pref)
search_results

[RealEstate(neighborhood='Buckhead, Atlanta, GA', vector=FixedSizeList(dim=1536), price=800000, bedrooms=4, bathrooms=3.0, house_size=3500, description='Spacious family home with a gourmet kitchen, large backyard, and a bonus room.', neighborhood_description='Upscale neighborhood known for its luxury shopping, fine dining, and prestigious schools.'),
 RealEstate(neighborhood='The Woodlands, Houston, TX', vector=FixedSizeList(dim=1536), price=700000, bedrooms=4, bathrooms=3.5, house_size=3000, description='Spacious family home with a large backyard, gourmet kitchen, and a game room.', neighborhood_description='Master-planned community known for its wooded surroundings, top-rated schools, and outdoor recreational amenities.')]

## Test

In [57]:
customer_input = """
I want a modern two-bedroom apartment in the city.
The property should be close to my workplace, with vibrant nightlife and many restaurants and cafes nearby.
Essential amenities include a gym, a rooftop terrace, and high-speed internet.
The neighborhood should have easy access to metro stations and be bike-friendly.
I prefer a highly urban area with lots of activities and events.
"""

pref = augment_customer_preference(customer_input)
search_results = find_estate(pref)
search_results

Generated Response:
 Neighborhood: Downtown City Center
Price: $2500
Bedrooms: 2
Bathrooms: 2
House Size: 1000 sq ft
Description: Modern two-bedroom apartment in the heart of the city with a gym, rooftop terrace, and high-speed internet.
Neighborhood Description: Vibrant area with plenty of nightlife, restaurants, cafes, easy access to metro stations, and bike-friendly streets. Perfect for those who enjoy urban living and being close to various activities and events.


[RealEstate(neighborhood='Cherry Creek, Denver, CO', vector=FixedSizeList(dim=1536), price=1100000, bedrooms=4, bathrooms=3.0, house_size=2800, description='Modern townhouse with a rooftop deck, open floor plan, and a two-car garage.', neighborhood_description='Trendy neighborhood known for its upscale shopping, art galleries, and fine dining options.'),
 RealEstate(neighborhood='South Beach, Miami, FL', vector=FixedSizeList(dim=1536), price=1500000, bedrooms=1, bathrooms=1.5, house_size=1200, description='Modern condo with ocean views, high ceilings, and access to a rooftop pool.', neighborhood_description='Vibrant neighborhood known for its beautiful beaches, trendy nightlife, and Art Deco architecture.')]

In [58]:
customer_input = """
I desire a three-bedroom beachfront house with direct access to the beach and stunning ocean views.
The property should be close to facilities for water sports.
It should also include a private dock, a swimming pool, and a spacious deck.
I need easy access to a marina and coastal highways.
The ideal neighborhood is a coastal town with a relaxed atmosphere and good tourist facilities.
"""

pref = augment_customer_preference(customer_input)
print(pref)
search_results = find_estate(pref)
search_results

Generated Response:
 Neighborhood: Coastal town with a relaxed atmosphere and good tourist facilities
Price: $1,500,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,500 sq ft
Description: Three-bedroom beachfront house with direct access to the beach, stunning ocean views, private dock, swimming pool, spacious deck, and close proximity to water sports facilities.
Neighborhood Description: The coastal town offers a relaxed atmosphere with good tourist facilities, easy access to a marina, and coastal highways.
{'neighborhood': 'Coastal town with a relaxed atmosphere and good tourist facilities', 'price': '$1,500,000', 'bedrooms': 3, 'bathrooms': 2.0, 'house_size': '2,500 sq ft', 'description': 'Three-bedroom beachfront house with direct access to the beach, stunning ocean views, private dock, swimming pool, spacious deck, and close proximity to water sports facilities.', 'neighborhood_description': 'The coastal town offers a relaxed atmosphere with good tourist facilities, easy access to a mar

[RealEstate(neighborhood='Venice Beach, Los Angeles, CA', vector=FixedSizeList(dim=1536), price=1200000, bedrooms=2, bathrooms=1.0, house_size=1500, description='Charming beach bungalow with a private backyard and just steps away from the ocean.', neighborhood_description='Trendy neighborhood known for its bohemian vibe, eclectic shops, and vibrant street art scene.'),
 RealEstate(neighborhood='South Beach, Miami, FL', vector=FixedSizeList(dim=1536), price=1500000, bedrooms=1, bathrooms=1.5, house_size=1200, description='Modern condo with ocean views, high ceilings, and access to a rooftop pool.', neighborhood_description='Vibrant neighborhood known for its beautiful beaches, trendy nightlife, and Art Deco architecture.')]